In [1]:
%load_ext autoreload
%autoreload 2

%matplotlib inline

## The forward and backward passes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=4960)

In [2]:
#export
from exp.nb_01 import *

def get_data():
    path = datasets.download_data(MNIST_URL, ext='.gz')
    with gzip.open(path, 'rb') as f:
        ((x_train, y_train), (x_valid, y_valid), _) = pickle.load(f, encoding='latin-1')
    return map(tensor, (x_train,y_train,x_valid,y_valid))

def normalize(x, m, s): return (x-m)/s

In [3]:
x_train,y_train,x_valid,y_valid = get_data()

In [4]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(0.1304), tensor(0.3073))

In [5]:
x_train = normalize(x_train, train_mean, train_std)
# NB: Use training, not validation mean for validation set
x_valid = normalize(x_valid, train_mean, train_std)

In [6]:
train_mean,train_std = x_train.mean(),x_train.std()
train_mean,train_std

(tensor(-4.4429e-07), tensor(1.))

In [7]:
#export
def test_near_zero(a,tol=1e-3): assert a.abs()<tol, f"Near zero: {a}"

In [8]:
test_near_zero(x_train.mean())
test_near_zero(1-x_train.std())

In [9]:
n,m = x_train.shape
c = y_train.max()+1
n,m,c

(50000, 784, tensor(10))

## Foundations version

### Basic architecture

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [10]:
# num hidden
nh = 50

[Tinker practice](https://course.fast.ai/videos/?lesson=8&t=5255)

In [12]:
# simplified kaiming init / he init
w1 = torch.randn(m,nh)/math.sqrt(m) # m is number of input
b1 = torch.zeros(nh)
w2 = torch.randn(nh,1)/math.sqrt(nh)
b2 = torch.zeros(1)

In [13]:
test_near_zero(w1.mean())
test_near_zero(w1.std()-1/math.sqrt(m))

In [14]:
# This should be ~ (0,1) (mean,std)...
x_valid.mean(),x_valid.std()

(tensor(-0.0059), tensor(0.9924))

In [15]:
def lin(x, w, b): return x@w + b

In [16]:
t = lin(x_valid, w1, b1)

In [17]:
#...so should this, because we used kaiming init, which is designed to do this
t.mean(),t.std()

(tensor(0.0108), tensor(0.9717))

In [18]:
def relu(x): return x.clamp_min(0.)

In [19]:
t = relu(lin(x_valid, w1, b1))

In [20]:
#...actually it really should be this!
t.mean(),t.std()

(tensor(0.3886), tensor(0.5747))

From pytorch docs: `a: the negative slope of the rectifier used after this layer (0 for ReLU by default)`

$$\text{std} = \sqrt{\frac{2}{(1 + a^2) \times \text{fan_in}}}$$

This was introduced in the paper that described the Imagenet-winning approach from *He et al*: [Delving Deep into Rectifiers](https://arxiv.org/abs/1502.01852), which was also the first paper that claimed "super-human performance" on Imagenet (and, most importantly, it introduced resnets!)

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=5128)

In [24]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2/m)

In [25]:
w1.mean(),w1.std()

(tensor(0.0002), tensor(0.0506))

In [26]:
t = relu(lin(x_valid, w1, b1))
t.mean(),t.std()

(tensor(0.5186), tensor(0.8027))

In [27]:
#export
from torch.nn import init

In [28]:
w1 = torch.zeros(m,nh)
init.kaiming_normal_(w1, mode='fan_out')
t = relu(lin(x_valid, w1, b1))

In [30]:
init.kaiming_normal_??

In [31]:
w1.mean(),w1.std()

(tensor(0.0004), tensor(0.0506))

In [32]:
t.mean(),t.std()

(tensor(0.5774), tensor(0.8296))

In [33]:
w1.shape

torch.Size([784, 50])

In [34]:
import torch.nn

In [35]:
torch.nn.Linear(m,nh).weight.shape

torch.Size([50, 784])

In [36]:
torch.nn.Linear.forward??

In [ ]:
torch.nn.functional.linear??

In [ ]:
torch.nn.Conv2d??

In [1]:
torch.nn.modules.conv._ConvNd.reset_parameters??

Object `torch.nn.modules.conv._ConvNd.reset_parameters` not found.


In [ ]:
# what if...?
def relu(x): return x.clamp_min(0.) - 0.5

In [38]:
# kaiming init / he init for relu
w1 = torch.randn(m,nh)*math.sqrt(2./m )
t1 = relu(lin(x_valid, w1, b1))
t1.mean(),t1.std()

(tensor(0.5654), tensor(0.8352))

In [39]:
def model(xb):
    l1 = lin(xb, w1, b1)
    l2 = relu(l1)
    l3 = lin(l2, w2, b2)
    return l3

In [40]:
%timeit -n 10 _=model(x_valid)

1.55 ms ± 207 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [41]:
assert model(x_valid).shape==torch.Size([x_valid.shape[0],1])

### Loss function: MSE

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6372)

In [42]:
model(x_valid).shape

torch.Size([10000, 1])

We need `squeeze()` to get rid of that trailing (,1), in order to use `mse`. (Of course, `mse` is not a suitable loss function for multi-class classification; we'll use a better loss function soon. We'll use `mse` for now to keep things simple.)

In [43]:
#export
def mse(output, targ): return (output.squeeze(-1) - targ).pow(2).mean()

In [44]:
y_train,y_valid = y_train.float(),y_valid.float()

In [45]:
preds = model(x_train)

In [46]:
preds.shape

torch.Size([50000, 1])

In [47]:
mse(preds, y_train)

tensor(22.8626)

### Gradients and backward pass

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=6493)

In [49]:
def mse_grad(inp, targ): 
    # grad of loss with respect to output of previous layer
    inp.g = 2. * (inp.squeeze() - targ).unsqueeze(-1) / inp.shape[0] # inp.shape[0] correspoinding to mean()

In [50]:
def relu_grad(inp, out):
    # grad of relu with respect to input activations
    inp.g = (inp>0).float() * out.g

In [62]:
def lin_grad(inp, out, w, b):
    # grad of matmul with respect to input
    inp.g = out.g @ w.t()
    print(f"inp.shape:{inp.shape}, out.g.shape:{out.g.shape}")
    w.g = (inp.unsqueeze(-1) * out.g.unsqueeze(1)).sum(0) 
    print((inp.unsqueeze(-1) * out.g.unsqueeze(1)).shape)
    b.g = out.g.sum(0) # sum(0) because it was applied to every outputs

In [63]:
def forward_and_backward(inp, targ):
    # forward pass:
    l1 = inp @ w1 + b1
    l2 = relu(l1)
    out = l2 @ w2 + b2
    # we don't actually need the loss in backward!
    loss = mse(out, targ)
    
    # backward pass:
    mse_grad(out, targ)
    lin_grad(l2, out, w2, b2)
    relu_grad(l1, l2)
    lin_grad(inp, l1, w1, b1)

In [136]:
forward_and_backward(x_train, y_train)

inp.shape:torch.Size([50000, 50]), out.g.shape:torch.Size([50000, 1])
torch.Size([50000, 50, 1])
inp.shape:torch.Size([50000, 784]), out.g.shape:torch.Size([50000, 50])
torch.Size([50000, 784, 50])


In [137]:
# Save for testing against later
w1g = w1.g.clone()
w2g = w2.g.clone()
b1g = b1.g.clone()
b2g = b2.g.clone()
ig  = x_train.g.clone()

We cheat a little bit and use PyTorch autograd to check our results.

In [83]:
xt2 = x_train.clone().requires_grad_(True)
w12 = w1.clone().requires_grad_(True)
w22 = w2.clone().requires_grad_(True)
b12 = b1.clone().requires_grad_(True)
b22 = b2.clone().requires_grad_(True)

In [84]:
def forward(inp, targ):
    # forward pass:
    l1 = inp @ w12 + b12
    l2 = relu(l1)
    out = l2 @ w22 + b22
    # we don't actually need the loss in backward!
    return mse(out, targ)

In [85]:
loss = forward(xt2, y_train)

In [86]:
loss.backward()

In [87]:
test_near(w22.grad, w2g)
test_near(b22.grad, b2g)
test_near(w12.grad, w1g)
test_near(b12.grad, b1g)
test_near(xt2.grad, ig )

## Refactor model

### Layers as classes

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7112)

In [96]:
class Relu():
    def __call__(self, inp):
        self.inp = inp
        self.out = inp.clamp_min(0.)-0.5
        return self.out
    
    def backward(self): self.inp.g = (self.inp>0).float() * self.out.g

In [97]:
class Lin():
    def __init__(self, w, b): self.w,self.b = w,b
        
    def __call__(self, inp):
        self.inp = inp
        self.out = inp@self.w + self.b
        return self.out
    
    def backward(self):
        self.inp.g = self.out.g @ self.w.t()
        # Creating a giant outer product, just to sum it, is inefficient!
        self.w.g = (self.inp.unsqueeze(-1) * self.out.g.unsqueeze(1)).sum(0)
        self.b.g = self.out.g.sum(0)

In [98]:
class Mse():
    def __call__(self, inp, targ):
        self.inp = inp
        self.targ = targ
        self.out = (inp.squeeze() - targ).pow(2).mean()
        return self.out
    
    def backward(self):
        self.inp.g = 2. * (self.inp.squeeze() - self.targ).unsqueeze(-1) / self.targ.shape[0]

In [99]:
class Model():
    def __init__(self, w1, b1, w2, b2):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [100]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model(w1, b1, w2, b2)

In [101]:
%time loss = model(x_train, y_train)

CPU times: user 321 ms, sys: 103 ms, total: 424 ms
Wall time: 10.7 ms


In [102]:
%time model.backward()

CPU times: user 15.4 s, sys: 20.9 s, total: 36.4 s
Wall time: 2.36 s


In [140]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

### Module.forward()

In [104]:
class Module():
    def __call__(self, *args):
        self.args = args
        self.out = self.forward(*args)
        return self.out
    
    def forward(self): raise Exception('not implemented')
    def backward(self): self.bwd(self.out, *self.args)

In [105]:
class Relu(Module):
    def forward(self, inp): return inp.clamp_min(0.)-0.5
    def bwd(self, out, inp): inp.g = (inp>0).float() * out.g

In [125]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = torch.einsum("bi,bj->ij", inp, out.g)
        print(self.w.g.shape)
        self.b.g = out.g.sum(0)

In [126]:
class Mse(Module):
    def forward (self, inp, targ): return (inp.squeeze() - targ).pow(2).mean()
    def bwd(self, out, inp, targ): inp.g = 2*(inp.squeeze()-targ).unsqueeze(-1) / targ.shape[0]

In [127]:
class Model():
    def __init__(self):
        self.layers = [Lin(w1,b1), Relu(), Lin(w2,b2)]
        self.loss = Mse()
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x, targ)
    
    def backward(self):
        self.loss.backward()
        for l in reversed(self.layers): l.backward()

In [128]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [129]:
%time loss = model(x_train, y_train)

CPU times: user 503 ms, sys: 345 ms, total: 848 ms
Wall time: 21.4 ms


In [130]:
%time model.backward()

torch.Size([50, 1])
torch.Size([784, 50])
CPU times: user 1.79 s, sys: 1.63 s, total: 3.42 s
Wall time: 85.6 ms


In [144]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

AssertionError: near:
tensor([[ -5.1868],
        [ -1.0837],
        [ -0.7778],
        [ -4.7991],
        [ -8.1166],
        [ -5.4788],
        [ -1.8906],
        [ -0.9770],
        [ -0.6579],
        [ -6.9860],
        [-13.9221],
        [ -3.4149],
        [ -5.0904],
        [ -2.7665],
        [ -2.0241],
        [ -1.1308],
        [ -1.4142],
        [ -1.7973],
        [ -2.4434],
        [ -1.5103],
        [ -5.9314],
        [ -5.6647],
        [ -2.1614],
        [ -1.5153],
        [ -1.2087],
        [ -5.6084],
        [ -0.4424],
        [ -1.6821],
        [ -1.5934],
        [ -5.2021],
        [ -6.4174],
        [ -3.6676],
        [ -1.9724],
        [ -3.2874],
        [ -3.3486],
        [-10.0884],
        [ -1.0642],
        [ -1.2454],
        [ -4.8092],
        [ -6.0708],
        [-10.9773],
        [ -1.3623],
        [ -4.0228],
        [ -7.4436],
        [-14.1324],
        [ -3.7992],
        [ -5.2512],
        [ -1.9995],
        [ -9.7474]

### Without einsum

[Jump_to lesson 8 video](https://course.fast.ai/videos/?lesson=8&t=7484)

In [131]:
class Lin(Module):
    def __init__(self, w, b): self.w,self.b = w,b
        
    def forward(self, inp): return inp@self.w + self.b
    
    def bwd(self, out, inp):
        inp.g = out.g @ self.w.t()
        self.w.g = inp.t() @ out.g
        self.b.g = out.g.sum(0)

In [132]:
w1.g,b1.g,w2.g,b2.g = [None]*4
model = Model()

In [141]:
%time loss = model(x_train, y_train)

CPU times: user 650 ms, sys: 779 ms, total: 1.43 s
Wall time: 36.1 ms


In [142]:
%time model.backward()

CPU times: user 1.81 s, sys: 2.04 s, total: 3.85 s
Wall time: 96.7 ms


In [143]:
test_near(w2g, w2.g)
test_near(b2g, b2.g)
test_near(w1g, w1.g)
test_near(b1g, b1.g)
test_near(ig, x_train.g)

AssertionError: near:
tensor([[ -5.1868],
        [ -1.0837],
        [ -0.7778],
        [ -4.7991],
        [ -8.1166],
        [ -5.4788],
        [ -1.8906],
        [ -0.9770],
        [ -0.6579],
        [ -6.9860],
        [-13.9221],
        [ -3.4149],
        [ -5.0904],
        [ -2.7665],
        [ -2.0241],
        [ -1.1308],
        [ -1.4142],
        [ -1.7973],
        [ -2.4434],
        [ -1.5103],
        [ -5.9314],
        [ -5.6647],
        [ -2.1614],
        [ -1.5153],
        [ -1.2087],
        [ -5.6084],
        [ -0.4424],
        [ -1.6821],
        [ -1.5934],
        [ -5.2021],
        [ -6.4174],
        [ -3.6676],
        [ -1.9724],
        [ -3.2874],
        [ -3.3486],
        [-10.0884],
        [ -1.0642],
        [ -1.2454],
        [ -4.8092],
        [ -6.0708],
        [-10.9773],
        [ -1.3623],
        [ -4.0228],
        [ -7.4436],
        [-14.1324],
        [ -3.7992],
        [ -5.2512],
        [ -1.9995],
        [ -9.7474]

### nn.Linear and nn.Module

In [145]:
#export
from torch import nn

In [146]:
class Model(nn.Module):
    def __init__(self, n_in, nh, n_out):
        super().__init__()
        self.layers = [nn.Linear(n_in,nh), nn.ReLU(), nn.Linear(nh,n_out)]
        self.loss = mse
        
    def __call__(self, x, targ):
        for l in self.layers: x = l(x)
        return self.loss(x.squeeze(), targ)

In [147]:
model = Model(m, nh, 1)

In [148]:
%time loss = model(x_train, y_train)

CPU times: user 346 ms, sys: 47.5 ms, total: 393 ms
Wall time: 10.3 ms


In [149]:
%time loss.backward()

CPU times: user 846 ms, sys: 352 ms, total: 1.2 s
Wall time: 31.5 ms


## Export

In [150]:
!./notebook2script.py 02_fully_connected.ipynb

Converted 02_fully_connected.ipynb to exp/nb_02.py
